In [ ]:
import sys, os, datetime, h5py
import numpy as np
from scipy import io
import pandas as pd
from matplotlib import pyplot
import tensorflow as tf
from tensorflow.keras.models import Model, model_from_json
from tensorflow.keras.layers import Input,InputLayer, Dense,  Dropout, Activation, Concatenate, Lambda
#from tensorflow.python.keras.summary import merge
from tensorflow.keras.utils import plot_model, multi_gpu_model
from tensorflow.keras.callbacks import CSVLogger, LearningRateScheduler, ModelCheckpoint, EarlyStopping
from tensorflow.keras import backend as K
from tensorflow.keras import regularizers
from functools import partial, update_wrapper
from tensorflow.keras import optimizers

from sklearn.preprocessing import StandardScaler

from lifelines.utils import concordance_index

np.random.seed(np.random.randint(100000))

datadir = "/home/k1onoda/work/DeepSurvival_20191111".replace('/', os.sep)

isAAL = 0
withAge = 0
withMMSE = 0
permutation = 0

onlyMCI = 0;

Itr = 100

CVs = 10
DROPOUT_RATIO = 0.3
NB_EPOCH = 1000
BATCH_SIZE = 128
N_GPUS = 1
#MRI = 2 # 1:1.5T 2: 3.0T

if  isAAL == 0:
    subdir_name = "BN_Age" +str(withAge)+"_MMSE"+str(withMMSE)
else:
    subdir_name = "AAL_Age"+str(withAge)+"_MMSE"+str(withMMSE)

if permutation == 1:
    subdir_name = subdir_name + "_permutation"
if onlyMCI == 1:
    subdir_name = subdir_name + "_onlyMCI"
    
subdir = datadir + os.sep + subdir_name
os.makedirs(subdir, exist_ok = True)  

J = 13
FINAL1 = np.zeros([Itr,J])

for ii in range(Itr):
    
    print(ii)
    K.clear_session()
    os.chdir( datadir )
    data = io.loadmat("GMV_20191111.mat")
    gmv = np.array( data["GMV"], dtype = 'float32' )
    if isAAL == 1:
        data = io.loadmat("GMV_AAL_20191111.mat")
        gmv = np.array( data["GMV"], dtype = 'float32' )
    nROIs = gmv.shape[1]    
    
    data = io.loadmat("Demo_20191111.mat" )
    demo = np.array( data["Demo"], dtype = 'float32'  )   # database group mri subjectID age sex convert interval MMSE

    #mristrength = np.array([demo[:,2]/2]).T
    #gmv = np.concatenate([gmv,mristrength],axis=1) 
    if withAge==1:
        age = np.array([demo[:,4]/100]).T
        gmv = np.concatenate([gmv, age],axis=1)
    if withMMSE==1:
        mmse = np.array([demo[:,8]/30]).T
        gmv = np.concatenate([gmv, mmse],axis=1)
        gmv = np.delete(gmv, np.where(demo[:,8]==0), 0)
        demo = np.delete(demo, np.where(demo[:,8]==0), 0)
    
    if onlyMCI==1:
        gmv = np.delete(gmv, np.where(demo[:,1]==3), 0)
        demo = np.delete(demo, np.where(demo[:,1]==3), 0)
    
    n_features = gmv.shape[1]   
    nn = gmv.shape[0];
    cv = np.zeros([nn])
    
    converter = (demo[:,6] == 1)  
    n_converter = np.sum(converter)
    sample_converter = np.arange(n_converter)
    np.random.shuffle(sample_converter)
    cv_converter = np.array(sample_converter % CVs)
    cv[np.where(converter)] = cv_converter
    
    nonconverter = (demo[:,6] == 0)  
    n_nonconverter = np.sum(nonconverter)
    sample_nonconverter = np.arange(n_nonconverter)
    np.random.shuffle(sample_nonconverter)
    cv_nonconverter = np.array(sample_nonconverter % CVs)
    cv[np.where(nonconverter)] = cv_nonconverter
    
    target_train = (cv>1)
    target_vali = (cv==0)
    target_test = (cv==1)

    features_train = gmv[target_train,:]
    features_vali = gmv[target_vali,:];
    features_test = gmv[target_test,:];
    
    t = demo[:,7]
    e = demo[:,6]
    if  permutation == 1:
        np.random.shuffle(t)
        np.random.shuffle(e)

    #J = np.int( np.round( np.max(t) ) + 1 )#J = np.int(np.ceil(np.max(t)) + 1)

    T = np.round(t).astype('int64')#T = np.ceil(t).astype('int64')
    T[T==0] = 1
    E = np.zeros([nn,J])
    mask = np.ones([nn,J]).astype('float32')
    for num in range(nn):
        if e[num] == 1:
            E[num,T[num]:J] = 1
        if e[num] == 0:
            mask[num,T[num]+1:J] = 0
    
    E_train = E[target_train,:]
    E_vali = E[target_vali,:]
    E_test = E[target_test,:]
    
    T_test = T[target_test]
    e_test = e[target_test]
    
    mask_train = mask[target_train,:]
    mask_vali = mask[target_vali,:]
    mask_test = mask[target_test,:]

    todaydetail  =  datetime.datetime.today()
    os.chdir(subdir)
    logdir = subdir + os.sep + "log_DeepSurv_" + todaydetail.strftime("%Y%m%d_%H%M%S")
    os.makedirs(logdir, exist_ok = True)
    os.chdir(logdir)
    print(os.getcwd())
    experiment_name = 'deepsurv' 

    from tensorflow.python.client import device_lib
    device_lib.list_local_devices()

    with tf.device("/cpu:0"):

        def output_of_lambda(input_shape):
            shape = list(input_shape)
            return (shape[0], J)

        def weibull_cdf(parameters):
            m = parameters[:,0]
            s = tf.maximum( parameters[:,1], 0.001 )
            output_list = []
            for num in range( J ):
                Time   = tf.constant( num, dtype="float32")
                e_Time = tf.pow( Time, m )
                s_Time = tf.negative( tf.div( e_Time, s) )
                x = tf.subtract( tf.constant(1, dtype="float32") , tf.exp( s_Time ) ) # F(t) = 1 - exp(-(t-g)^m/s) #ref http://www.mogami.com/notes/weibull.html
                output_list.append ( x )
            return tf.stack(output_list, axis=1)

        def generator_loss(y_true, y_pred, weights):  # y_true's shape=(batch_size, row, col, ch)
            #loss = tf.cumsum( tf.multiply( tf.square( tf.subtract( y_pred, y_true ) ), weights ), axis=1, reverse=True)[:,0]
            log_p = tf.log( tf.add( y_pred,  tf.constant(1.0) ) )
            log_t = tf.log( tf.add( y_true,  tf.constant(1.0) ) )
            loss = tf.cumsum( tf.multiply( tf.square( tf.subtract( log_p, log_t ) ), weights ), axis=1, reverse=True)[:,0]
            return loss

        def wrapped_generator_loss(func, *args, **kwargs):
            partial_generator_loss = partial(generator_loss, *args, **kwargs)
            update_wrapper(partial_generator_loss, generator_loss)
            return partial_generator_loss

        inputs = Input((n_features,), name='inputs')
        x1 = Dense(units=32, activation='relu', name='hidden_layer1',
                    kernel_regularizer=regularizers.l1_l2(0.001))(inputs)
        x1 = Dropout(DROPOUT_RATIO)(x1)
        x2 = Dense(units=32, activation='relu', name='hidden_layer2',
                    kernel_regularizer=regularizers.l1_l2(0.001))(x1)
        x2 = Dropout(DROPOUT_RATIO)(x2)
        x3 = Dense(units=32, activation='relu', name='hidden_layer3',
                    kernel_regularizer=regularizers.l1_l2(0.001))(x2)
        x3 = Dropout(DROPOUT_RATIO)(x3)
        p1 = Dense(units=1, activation='softplus', name='param1_layer')(x3)
        p2 = Dense(units=1, activation='relu', name='param2_layer')(x3)
        parameters = Concatenate(name='params_layer')([p1, p2])
        y_pred = Lambda(weibull_cdf, output_shape=output_of_lambda)(parameters)

        mask_batch = Input((J,), name='mask_bartch')
        L = wrapped_generator_loss(generator_loss, weights = mask_batch)

        model = Model(inputs= [inputs, mask_batch], outputs = y_pred)
        #model.summary()
    
    if  N_GPUS>=2:
        models = multi_gpu_model(model, gpus=N_GPUS)
    else:
        models = model

    pred_params = np.zeros([nn,2])
    c_index_test = np.zeros([J,1])

    scaler = StandardScaler()
    scaler.fit(features_train)
    features_train = scaler.transform(features_train)
    features_vali = scaler.transform(features_vali)
    features_test = scaler.transform(features_test)

    outputfilename     = 'Training.csv'
    weightfilename     = 'WeightBest.h5'

    checkpointer = ModelCheckpoint(filepath=weightfilename, monitor='loss', verbose=1, save_best_only=True)
    early_stopping = EarlyStopping(monitor='val_loss',patience=10,verbose=1)
    callbacks = []
    callbacks.append(early_stopping)
    callbacks.append(CSVLogger(outputfilename))
    #callbacks.append(checkpointer)

    adm = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    models.compile(optimizer=adm, loss=L)
    #models.compile(optimizer='Adam', loss='mean_squared_error', metrics=["accuracy"])
    session = K.get_session()

    models.fit([features_train, mask_train], E_train, batch_size=BATCH_SIZE, epochs = NB_EPOCH, callbacks=callbacks, verbose=0, validation_data = ([features_vali, mask_vali], E_vali))
    model.save_weights('Model_Weights_' + str(num+1) + '.h5')

    #intermediate_model = Model(inputs=model.input, outputs=model.get_layer('params_layer').output)

    #models.load_weights(weightfilename)
    #models.compile(optimizer='Adam', loss=L)
    prob = models.predict([features_test, mask_test], batch_size=BATCH_SIZE, verbose=1)
    intermediate_model = Model(inputs=model.input, outputs=model.get_layer('params_layer').output)
    pred_params = intermediate_model.predict([features_test, mask_test])

    predictionfilename = 'Param.csv'
    prediction = np.c_[demo[target_test,:],pred_params, prob]
    np.savetxt(predictionfilename, prediction, delimiter=',')

    c_index = np.zeros([J])
    for num in range(J - 1) :
        c_index[num+1] = concordance_index(np.array(T_test),1/prob[:,num+1], np.array(e_test))
    print( c_index )

    cindexfilename = 'C_index.txt'
    np.savetxt(cindexfilename, c_index)
    
    FINAL1[ii,:] = c_index.T

    #json_string = models.to_json()
    #modeltxtfilename   = 'Modeltxt_' + 'Demo.txt'
    #f = open(modeltxtfilename,'w')
    #f.write(json_string)
    #f.close()

col_header1 = []
for t in range(12):
    col_header1.append(str(t+1) + 'yr c_index')
    
df_ci = pd.DataFrame(FINAL1[:,1:J], columns=col_header1)
df_ci.to_csv(subdir + '/result_CINDEX.csv')
print(np.mean(np.mean(FINAL1,axis=1),axis=0))    
